In [1]:
import numpy as np
from pathlib import Path
import pandas as pd
from tqdm import tqdm

# --- 1. CONFIGURATION ---
PROJECT_PATH = Path('/content/drive/My Drive/AR_Downscaling')
DATA_DIR = PROJECT_PATH / 'final_dataset_multi_variable'
OUTPUT_DIR = PROJECT_PATH / 'training_data_stats'
OUTPUT_DIR.mkdir(exist_ok=True)

# Thresholds to Analyze
THRESHOLDS_K = [230.0, 220.0, 210.0]

# Dataset splits to analyze
SPLITS = ["train", "val", "test"]

def analyze_event_rarity():
    all_split_summaries = []

    for split in SPLITS:
        print(f"\n🔬 Analyzing split: {split.upper()}")

        split_dir = DATA_DIR / split
        target_files = sorted(list(split_dir.glob('*_target.npy')))
        if not target_files:
            print(f"❌ No target files found in {split_dir}. Skipping.")
            continue

        print(f"Found {len(target_files)} {split} samples to analyze.")

        # Store the frequency for each sample
        all_frequencies = []
        for target_file in tqdm(target_files, desc=f"Processing {split} samples"):
            target_k = np.load(target_file).astype(np.float32)
            total_pixels = target_k.size
            sample_freq = {'filename': target_file.name}

            for thr in THRESHOLDS_K:
                pixels_below_threshold = (target_k <= thr).sum()
                frequency = (pixels_below_threshold / total_pixels) * 100
                sample_freq[f'freq_{int(thr)}K'] = frequency

            all_frequencies.append(sample_freq)

        if not all_frequencies:
            continue

        # Convert results to DataFrame
        df = pd.DataFrame(all_frequencies)

        print("\n📊 FINAL EVENT RARITY REPORT for", split.upper())
        for thr in THRESHOLDS_K:
            col = f'freq_{int(thr)}K'
            mean_freq = df[col].mean()
            std_freq = df[col].std()
            print(f"  - T <= {int(thr)}K: {mean_freq:.4f}% (±{std_freq:.4f}%)")

        # Save per-sample results
        per_sample_path = OUTPUT_DIR / f'event_rarity_statistics_{split}.csv'
        df.to_csv(per_sample_path, index=False)
        print(f"💾 Detailed per-sample results saved to: {per_sample_path}")

        # Save summary stats for this split
        for thr in THRESHOLDS_K:
            col = f'freq_{int(thr)}K'
            all_split_summaries.append({
                'split': split,
                'threshold': thr,
                'mean_%': df[col].mean(),
                'std_%': df[col].std()
            })

    # Combine and save all splits summary
    if all_split_summaries:
        summary_df = pd.DataFrame(all_split_summaries)
        summary_path = OUTPUT_DIR / 'event_rarity_summary_all.csv'
        summary_df.to_csv(summary_path, index=False)
        print(f"\n✅ Combined summary for all splits saved to: {summary_path}")

if __name__ == "__main__":
    try:
        from google.colab import drive
        drive.mount('/content/drive', force_remount=True)
    except ImportError:
        print("Not running in Google Colab. Assuming local file paths.")

    analyze_event_rarity()


Mounted at /content/drive

🔬 Analyzing split: TRAIN
Found 1200 train samples to analyze.


Processing train samples: 100%|██████████| 1200/1200 [02:41<00:00,  7.42it/s]



📊 FINAL EVENT RARITY REPORT for TRAIN
  - T <= 230K: 46.2223% (±34.4570%)
  - T <= 220K: 18.7507% (±19.0172%)
  - T <= 210K: 6.6292% (±9.1181%)
💾 Detailed per-sample results saved to: /content/drive/My Drive/AR_Downscaling/training_data_stats/event_rarity_statistics_train.csv

🔬 Analyzing split: VAL
Found 150 val samples to analyze.


Processing val samples: 100%|██████████| 150/150 [00:09<00:00, 15.52it/s]



📊 FINAL EVENT RARITY REPORT for VAL
  - T <= 230K: 34.5756% (±34.5958%)
  - T <= 220K: 13.9523% (±19.7722%)
  - T <= 210K: 4.2478% (±8.0255%)
💾 Detailed per-sample results saved to: /content/drive/My Drive/AR_Downscaling/training_data_stats/event_rarity_statistics_val.csv

🔬 Analyzing split: TEST
Found 150 test samples to analyze.


Processing test samples: 100%|██████████| 150/150 [00:08<00:00, 16.78it/s]


📊 FINAL EVENT RARITY REPORT for TEST
  - T <= 230K: 49.2168% (±32.6427%)
  - T <= 220K: 19.6215% (±19.2401%)
  - T <= 210K: 6.2629% (±9.1381%)
💾 Detailed per-sample results saved to: /content/drive/My Drive/AR_Downscaling/training_data_stats/event_rarity_statistics_test.csv

✅ Combined summary for all splits saved to: /content/drive/My Drive/AR_Downscaling/training_data_stats/event_rarity_summary_all.csv
